# Testing alternatives to the CSF function
 also testing some regressions and parametric distributions

In [11]:
# Import Libraries - Enhanced for Statistical Analysis
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Set Up Database Connection
# Replace the placeholders with your actual database credentials
DB_CONFIG = {
    "dbname": "csgo_parsed",
    "user": "csgo_parser",
    "password": "3?6B7yTGPrkJF34p",
    "host": "192.168.1.100",
    "port": "5444"
}

In [12]:


# Ensure we have a fresh connection
try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("✅ Database connection established")
except Exception as e:
    print(f"❌ Connection error: {e}")


print("📊 Helper functions defined successfully!")

✅ Database connection established
📊 Helper functions defined successfully!


In [13]:
# Query for detailed round outcomes
query_detailed = """
WITH round_equipment AS (
    SELECT 
        r.id,
        r.match_id,
        r.id_demo_exports,
        r.round_num,
        r.ct_winner,
        r.team1_winner,
        r.is_ct_t1,
        r.round_end_reason,
        -- CT team equipment
        SUM(CASE WHEN (is_ct_t1 = (pr.team = 1)) THEN pr.eq_val_fte ELSE 0 END) as ct_equipment,
        -- T team equipment
        SUM(CASE WHEN (is_ct_t1 != (pr.team = 1)) THEN pr.eq_val_fte ELSE 0 END) as t_equipment,
        hmi.team_1_id,
        hmi.team_2_id,
        hmi.event_id
    FROM rounds_ed r
    JOIN player_round_ed pr ON r.id = pr.round_id
    LEFT JOIN hltv_match_info hmi ON r.match_id = hmi.match_id
    WHERE pr.eq_val_fte > 0
    GROUP BY 1,2,3,4,5,6,7,8, 11,12,13
),
bomb_planted AS (
    SELECT 
        round_id,
        CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END as bomb_planted
    FROM bomb_events_round_ed
    WHERE bomb_event_type = 0
    GROUP BY round_id
),
round_survivors AS (
    SELECT 
        r.id as round_id,
        r.is_ct_t1,
        -- Count total players per team (5 players each)
        -- CT survivors = 5 - (deaths of CT players)
        5 - COUNT(DISTINCT CASE 
            WHEN (r.is_ct_t1 = (pv.team = 1)) THEN k.victim_hltv_id 
        END) as ct_survivors,
        -- T survivors = 5 - (deaths of T players)
        5 - COUNT(DISTINCT CASE 
            WHEN (r.is_ct_t1 != (pv.team = 1)) THEN k.victim_hltv_id 
        END) as t_survivors
    FROM rounds_ed r
    LEFT JOIN kills_round_ed k ON r.id = k.round_id
    LEFT JOIN player_round_ed pv ON k.victim_hltv_id = pv.player_id AND k.round_id = pv.round_id and pv.player_id != 0
    GROUP BY 1,2
),
saved_equipment AS (
    SELECT 
        r.id as round_id,
        r.is_ct_t1,
        -- CT team saved equipment from player_economy_ed joined with player_round_ed
        SUM(CASE WHEN (r.is_ct_t1 = (pr.team = 1)) THEN COALESCE(pe.saved_eq_val, 0) ELSE 0 END) as ct_saved_equipment,
        -- T team saved equipment from player_economy_ed joined with player_round_ed
        SUM(CASE WHEN (r.is_ct_t1 != (pr.team = 1)) THEN COALESCE(pe.saved_eq_val, 0) ELSE 0 END) as t_saved_equipment
    FROM rounds_ed r
    JOIN player_economy_ed pe ON r.id = pe.round_id
    JOIN player_round_ed pr ON r.id = pr.round_id AND pe.player_id = pr.player_id and pr.player_id != 0
    GROUP BY 1,2
),
player_money AS (
    SELECT 
        r.id as round_id,
        r.is_ct_t1,
        -- CT team player money (money_earned + money_earned_re) joined with player_round_ed
        SUM(CASE WHEN (r.is_ct_t1 = (pr.team = 1)) 
            THEN COALESCE(pe.money_earned, 0)
            ELSE 0 END) as ct_player_money,
        -- T team player money (money_earned + money_earned_re) joined with player_round_ed
        SUM(CASE WHEN (r.is_ct_t1 != (pr.team = 1)) 
            THEN COALESCE(pe.money_earned, 0)  
            ELSE 0 END) as t_player_money
    FROM rounds_ed r
    JOIN player_economy_ed pe ON r.id = pe.round_id
    JOIN player_round_ed pr ON r.id = pr.round_id AND pe.player_id = pr.player_id and pr.player_id != 0
    GROUP BY 1,2
), 
ranked_data AS (
    SELECT 
        trd.id,
        trd.id_demo_exports,
        trd.round_num,
        trd.team1_winner,
        -- Team 1 metrics
        CAST(het1.rank_during AS INTEGER) as t1_rank,
        -- Team 2 metrics
        CAST(het2.rank_during AS INTEGER) as t2_rank
    FROM round_equipment trd
    LEFT JOIN hltv_events_teams het1 ON het1.team_id = trd.team_1_id AND het1.event_id = trd.event_id
    LEFT JOIN hltv_events_teams het2 ON het2.team_id = trd.team_2_id AND het2.event_id = trd.event_id
    WHERE het1.rank_during IS NOT NULL 
        AND het2.rank_during IS NOT NULL
)
SELECT 
    re.*,
    -- Add bomb planted status
    COALESCE(bp.bomb_planted, 0) as bomb_planted,
    -- Add survivor counts from round_survivors CTE
    rs.ct_survivors,
    rs.t_survivors,
    -- Higher and lower equipment
    CASE 
        WHEN re.ct_equipment > re.t_equipment THEN re.ct_equipment
        ELSE re.t_equipment
    END as higher_equipment,
    CASE 
        WHEN re.ct_equipment <= re.t_equipment THEN re.ct_equipment
        ELSE re.t_equipment
    END as lower_equipment,
    -- Did higher equipment team win?
    CASE 
        WHEN (re.ct_equipment > re.t_equipment AND re.ct_winner)
        OR (re.ct_equipment < re.t_equipment AND NOT re.ct_winner)
        THEN 1
        ELSE 0
    END as higher_eq_won,
    -- Winner's survivors (only from is_alive_re)
    CASE 
        WHEN re.ct_winner THEN rs.ct_survivors
        ELSE rs.t_survivors
    END as winner_survivors,
    -- Winner's saved equipment (from player_economy_ed.saved_eq_val)
    CASE 
        WHEN re.ct_winner THEN se.ct_saved_equipment
        ELSE se.t_saved_equipment
    END as winner_saved_equipment,
    -- CT and T saved equipment 
    se.ct_saved_equipment,
    se.t_saved_equipment,
    -- All money columns for analysis
    pm.ct_player_money,
    pm.t_player_money,
    rd.t1_rank,
    rd.t2_rank
FROM round_equipment re
LEFT JOIN bomb_planted bp ON re.id = bp.round_id
LEFT JOIN round_survivors rs ON re.id = rs.round_id
LEFT JOIN saved_equipment se ON re.id = se.round_id
LEFT JOIN player_money pm ON re.id = pm.round_id
LEFT JOIN ranked_data rd ON re.id = rd.id
WHERE re.ct_equipment > 0 AND re.t_equipment > 0 
ORDER BY re.match_id, re.round_num
"""

print("📊 Fetching detailed round data...")
detailed_df = pd.read_sql_query(query_detailed, conn)
print(f"✅ Loaded {len(detailed_df):,} rounds with complete data")
print()

# Combine round end reasons 12 and 7
print("🔹 Pre-processing: Combining round end reasons 12 and 7")
detailed_df['round_end_reason'] = detailed_df['round_end_reason'].replace({12: 7})
print(f"   Round end reasons 12 merged into 7")
print()

# Recode round end reasons: 1→1, 9→2, 7→3, 8→4
print("🔹 Recoding round end reasons")
original_count = len(detailed_df)
print(f"   Original distribution:")
for old_code in [1, 7, 8, 9]:
    count = (detailed_df['round_end_reason'] == old_code).sum()
    pct = count / len(detailed_df) * 100 if len(detailed_df) > 0 else 0
    print(f"      Reason {old_code}: {count:,} rounds ({pct:.2f}%)")

detailed_df['round_end_reason'] = detailed_df['round_end_reason'].replace({
    1: 1,  # T Win (Target Bombed) - no change
    9: 2,  # T Win (Elimination) - 9 → 2
    7: 3,  # CT Win (Defuse) - 7 → 3
    8: 4,  # CT Win (Elimination) - 8 → 4
})

print(f"   New distribution:")
for new_code in [1, 2, 3, 4]:
    count = (detailed_df['round_end_reason'] == new_code).sum()
    pct = count / len(detailed_df) * 100 if len(detailed_df) > 0 else 0
    print(f"      Reason {new_code}: {count:,} rounds ({pct:.2f}%)")
print()



📊 Fetching detailed round data...


C:\Users\peter\AppData\Local\Temp\ipykernel_6092\2205888952.py:150: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  detailed_df = pd.read_sql_query(query_detailed, conn)


✅ Loaded 2,601,592 rounds with complete data

🔹 Pre-processing: Combining round end reasons 12 and 7
   Round end reasons 12 merged into 7

🔹 Recoding round end reasons
   Original distribution:
      Reason 1: 407,446 rounds (15.66%)
      Reason 7: 405,067 rounds (15.57%)
      Reason 8: 934,538 rounds (35.92%)
      Reason 9: 854,541 rounds (32.85%)
   New distribution:
      Reason 1: 407,446 rounds (15.66%)
      Reason 2: 854,541 rounds (32.85%)
      Reason 3: 405,067 rounds (15.57%)
      Reason 4: 934,538 rounds (35.92%)



In [14]:


# Filter out inconsistent data
# Reason 2 (T Win Elimination - was 9) should have CT survivors = 0
filtered_9 = (detailed_df['round_end_reason'] == 2) & (detailed_df['ct_survivors'] > 0)
reason_9_issues = detailed_df[filtered_9]
print(f"🔹 Filtering inconsistent Reason 2 (T Win Elimination) rounds:")
print(f"   Found {len(reason_9_issues):,} rounds where Reason 2 occurred but CT had survivors > 0")

before = len(detailed_df)
detailed_df = detailed_df[~filtered_9]
total_filtered = before - len(detailed_df)

print(f"   Removed: {total_filtered:,} rounds")
print(f"   Remaining: {len(detailed_df):,} rounds")
print()
print("✅ Data quality filtering complete")

🔹 Filtering inconsistent Reason 2 (T Win Elimination) rounds:
   Found 2,821 rounds where Reason 2 occurred but CT had survivors > 0
   Removed: 2,821 rounds
   Remaining: 2,598,771 rounds

✅ Data quality filtering complete


# The Data Science/ Data Engineering part


In [15]:
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score, roc_curve, auc

import statsmodels.api as sm

test_size_global = 0.3
random_state_global = 42

In [16]:
detailed_df['ct_t_ratio'] = np.log(detailed_df['ct_equipment'] / detailed_df['t_equipment'])
x_r = detailed_df[['ct_t_ratio']]
y = detailed_df['ct_winner']

# Split data
xtr_train, xtr_test, ytr_train, ytr_test = train_test_split(x_r, y, test_size=test_size_global, random_state=random_state_global)
# train dataset
log_model = sm.Logit(ytr_train, sm.add_constant(xtr_train))
result = log_model.fit()
print(result.summary())

# test dataset
yhat = result.predict(sm.add_constant(xtr_test))
prediction = list(map(round, yhat))

cm = confusion_matrix(ytr_test, prediction) 
auc_score = roc_auc_score(ytr_test, yhat)
fpr, tpr, _ = roc_curve(ytr_test, yhat)

print ("Confusion Matrix : \n", cm) 
print('Test accuracy = ', accuracy_score(ytr_test, prediction))
print('Test AUC-ROC = ', auc_score) 


Optimization terminated successfully.
         Current function value: 0.582403
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              ct_winner   No. Observations:              1819139
Model:                          Logit   Df Residuals:                  1819137
Method:                           MLE   Df Model:                            1
Date:                Tue, 20 Jan 2026   Pseudo R-squ.:                  0.1592
Time:                        17:57:28   Log-Likelihood:            -1.0595e+06
converged:                       True   LL-Null:                   -1.2601e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0021      0.002     -1.255      0.210      -0.005       0.001
ct_t_ratio     1.0834      0.

In [17]:
x = np.log(detailed_df[['ct_equipment', 't_equipment']])
y = detailed_df['ct_winner']

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_global, random_state=random_state_global)

# train dataset
log2_model = sm.Logit(y_train, sm.add_constant(x_train))
result2 = log2_model.fit()
print(result2.summary())

# test dataset
yhat2 = result2.predict(sm.add_constant(x_test))
prediction = list(map(round, yhat2))

cm = confusion_matrix(y_test, prediction) 
auc_score = roc_auc_score(ytr_test, yhat2)
fpr, tpr, _ = roc_curve(ytr_test, yhat2)

print ("Confusion Matrix : \n", cm) 
print('Test accuracy = ', accuracy_score(ytr_test, prediction))
print('Test AUC-ROC = ', auc_score)

Optimization terminated successfully.
         Current function value: 0.582394
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              ct_winner   No. Observations:              1819139
Model:                          Logit   Df Residuals:                  1819136
Method:                           MLE   Df Model:                            2
Date:                Tue, 20 Jan 2026   Pseudo R-squ.:                  0.1592
Time:                        17:57:38   Log-Likelihood:            -1.0595e+06
converged:                       True   LL-Null:                   -1.2601e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1592      0.028     -5.750      0.000      -0.214      -0.105
ct_equipment     1.0902

In [18]:
from scipy import stats as scipy_stats

# Add difference in rank to the regression
detailed_df = detailed_df.dropna(subset=['t1_rank', 't2_rank'])
detailed_df['ct_rank'] = detailed_df.apply(lambda row: row['t1_rank'] if row['is_ct_t1'] else row['t2_rank'], axis=1)
detailed_df['t_rank'] = detailed_df.apply(lambda row: row['t2_rank'] if row['is_ct_t1'] else row['t1_rank'], axis=1)
detailed_df['rank_diff'] = detailed_df['ct_rank'] - detailed_df['t_rank']
detailed_df['ct_t_ratio'] = np.log(detailed_df['ct_equipment'] / detailed_df['t_equipment'])

# Filter out rows with missing rank_diff values
df_with_ranks = detailed_df[['ct_t_ratio', 'rank_diff', 'ct_winner']].dropna()
X = df_with_ranks[['ct_t_ratio', 'rank_diff']].copy()
y = df_with_ranks['ct_winner']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_global, random_state=random_state_global)

# Fit logistic regression with statsmodels
logit_model = sm.Logit(y_train, sm.add_constant(X_train))
result = logit_model.fit()
print(result.summary())

# Predict and evaluate
yhat = result.predict(sm.add_constant(X_test))
prediction = list(map(round, yhat))

cm = confusion_matrix(y_test, prediction)
auc_score = roc_auc_score(y_test, yhat)
fpr, tpr, _ = roc_curve(y_test, yhat)

print("Confusion Matrix : \n", cm)
print('Test accuracy = ', accuracy_score(y_test, prediction))
print('Test AUC-ROC = ', auc_score)

# Perform Wald test for rank_diff coefficient

# Get the coefficient and standard error for rank_diff
rank_diff_coef = result.params['rank_diff']
rank_diff_se = result.bse['rank_diff']

# Calculate Wald statistic: (coefficient / std_error)^2
wald_stat = (rank_diff_coef / rank_diff_se) ** 2

# Calculate p-value (chi-square distribution with 1 degree of freedom)
p_value = 1 - scipy_stats.chi2.cdf(wald_stat, df=1)

print("\n" + "="*50)
print("Wald Test for rank_diff")
print("="*50)
print(f"Coefficient: {rank_diff_coef:.6f}")
print(f"Standard Error: {rank_diff_se:.6f}")
print(f"Wald Statistic: {wald_stat:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Significance: {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'Not significant'}")

Optimization terminated successfully.
         Current function value: 0.582606
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              ct_winner   No. Observations:              1274806
Model:                          Logit   Df Residuals:                  1274803
Method:                           MLE   Df Model:                            2
Date:                Tue, 20 Jan 2026   Pseudo R-squ.:                  0.1588
Time:                        17:58:49   Log-Likelihood:            -7.4271e+05
converged:                       True   LL-Null:                   -8.8295e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0047      0.002     -2.354      0.019      -0.009      -0.001
ct_t_ratio     1.0842      0.

In [19]:
x = np.log(detailed_df[['ct_equipment', 't_equipment', 'ct_rank', 't_rank']])
y = detailed_df['ct_winner']

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_global, random_state=random_state_global)

# train dataset
log2_model = sm.Logit(y_train, sm.add_constant(x_train))
result2 = log2_model.fit()
print(result2.summary())

# test dataset
yhat2 = result2.predict(sm.add_constant(x_test))
prediction = list(map(round, yhat2))

cm = confusion_matrix(y_test, prediction) 
auc_score = roc_auc_score(y_test, yhat2)
fpr, tpr, _ = roc_curve(y_test, yhat2)

print ("Confusion Matrix : \n", cm) 
print('Test accuracy = ', accuracy_score(y_test, prediction))
print('Test AUC-ROC = ', auc_score) 

Optimization terminated successfully.
         Current function value: 0.582566
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              ct_winner   No. Observations:              1274806
Model:                          Logit   Df Residuals:                  1274801
Method:                           MLE   Df Model:                            4
Date:                Tue, 20 Jan 2026   Pseudo R-squ.:                  0.1589
Time:                        17:58:56   Log-Likelihood:            -7.4266e+05
converged:                       True   LL-Null:                   -8.8295e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1578      0.034     -4.576      0.000      -0.225      -0.090
ct_equipment     1.0908

In [20]:
from scipy import stats as scipy_stats

# Add difference in rank to the regression
detailed_df = detailed_df.dropna(subset=['t1_rank', 't2_rank'])
detailed_df['rank_log_diff'] = np.log(detailed_df['ct_rank']) - np.log(detailed_df['t_rank'])
detailed_df['eq_log_diff'] = np.log(detailed_df['ct_equipment']) - np.log(detailed_df['t_equipment'])

# Filter out rows with missing rank_diff values
df_with_ranks = detailed_df[['rank_log_diff', 'eq_log_diff', 'ct_winner']].dropna()
X = df_with_ranks[['rank_log_diff', 'eq_log_diff']].copy()
y = df_with_ranks['ct_winner']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_global, random_state=random_state_global)

# Fit logistic regression with statsmodels
logit_model = sm.Logit(y_train, sm.add_constant(X_train))
result = logit_model.fit()
print(result.summary())

# Predict and evaluate
yhat = result.predict(sm.add_constant(X_test))
prediction = list(map(round, yhat))

cm = confusion_matrix(y_test, prediction)
auc_score = roc_auc_score(y_test, yhat)
fpr, tpr, _ = roc_curve(y_test, yhat)

print("Confusion Matrix : \n", cm)
print('Test accuracy = ', accuracy_score(y_test, prediction))
print('Test AUC-ROC = ', auc_score)

# Extract coefficients and p-values
print("\n" + "="*50)
print("Model Coefficients and P-values")
print("="*50)
for var in ['rank_log_diff', 'eq_log_diff']:
    coef = result.params[var]
    pval = result.pvalues[var]
    se = result.bse[var]
    print(f"\n{var}:")
    print(f"  Coefficient: {coef:.6f}")
    print(f"  Std Error: {se:.6f}")
    print(f"  P-value: {pval:.6f}")
    print(f"  Significance: {'***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else 'Not significant'}")

Optimization terminated successfully.
         Current function value: 0.582574
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              ct_winner   No. Observations:              1274806
Model:                          Logit   Df Residuals:                  1274803
Method:                           MLE   Df Model:                            2
Date:                Tue, 20 Jan 2026   Pseudo R-squ.:                  0.1589
Time:                        17:59:03   Log-Likelihood:            -7.4267e+05
converged:                       True   LL-Null:                   -8.8295e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0045      0.002     -2.290      0.022      -0.008      -0.001
rank_log_diff    -0.